In [1]:
import sys
import os
sys.path.append(os.path.abspath("..")) 

import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import Column, Integer, String, DateTime, func
from sqlalchemy.orm import declarative_base
from sqlalchemy import inspect
from sqlalchemy import text
from database import engine

import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

from database import Base, SessionLocal, engine, ensure_views_from_files, init_db
from main.sql import load_dict, count_overlap_word

In [2]:
init_db()
ensure_views_from_files()

with engine.begin() as conn:
    conn.execute(text("DROP TABLE IF EXISTS TranslationLog"))

## Downloading Quiz Score

In [3]:
df = load_dict()

In [4]:
df.head()

,Word Id,Word,Pinyin,Pinyin Simplified,Meaning,Added Date,Word Category,Word Rarity,Type,Sentence,Sentence Pinyin,Sentence Meaning,Quiz Attempts,Num Pinyin Correct,Num Meaning Correct,Last Quiz
0,D000002,包裹,bao1 guo3,bao1 guo3,Package,2024-09-21 00:00:00.000000,Object,Common,Noun,他昨天收到了一个包裹。,Wǒ zuótiān shōudào le bāoguǒ.,I received the package yesterday.,2,0,0,2026-01-27 00:00:00.000000
1,D000004,不错,bu2 cuo4,bu2 cuo4,Good (More positive than 还好),2024-09-21 00:00:00.000000,Opinion,Common,Adjective,这个电影不错，我们可以一起去看。,Zhè gè diànyǐng bùcuò wǒmen kěyǐ yìqǐ qù kàn,This movie is pretty good; we can go watch it ...,0,0,0,None
2,D000005,不过,bu2 guo4,bu2 guo4,But (Similar to dan4 shi4 but less formal),2024-09-21 00:00:00.000000,Grammar,Common,Grammar,我喜欢这家餐厅，不过价格有点贵。,Wǒ xǐhuān zhè jiā cāntīng búguò jiàgé yǒudiǎn ...,I like this restaurant but the price is a bit ...,0,0,0,None
3,D000006,不太,bu2 tai4,bu2 tai4,not quite,2024-09-21 00:00:00.000000,Degree,Common,Adjuster,不太好,bu2 tai4 hao3,not quite good,0,0,0,None
4,D000007,不仅,bu4 jin3,bu4 jin3,Not only ....,2024-09-21 00:00:00.000000,Grammar,Common,Grammar,他不仅喜欢音乐，还喜欢运动,Tā bùjǐn xǐhuān yīnyuè hái xǐhuān yùndòng.,He not only likes music but also likes sports.,0,0,0,None


In [5]:
df['Num Pinyin Wrong'] = df['Quiz Attempts'] - df['Num Pinyin Correct']
df['Num Meaning Wrong'] = df['Quiz Attempts'] - df['Num Meaning Correct']
df['Last Quiz'] = pd.to_datetime(df['Last Quiz'], format='mixed', errors='coerce')

df['Right Score'] = (df['Num Pinyin Correct'] + df['Num Meaning Correct']) / (2 * df['Quiz Attempts']) * 100
df['Wrong Score'] = 100 - df['Right Score']

In [6]:
score_summary_word = df.groupby('Word')[['Right Score', 'Wrong Score', 'Num Pinyin Wrong', 'Num Meaning Wrong']].sum().reset_index()
top_10_pinyin_wrong = score_summary_word.loc[score_summary_word['Num Pinyin Wrong']>=1].sort_values(by='Num Pinyin Wrong', ascending=False)['Word'].to_list()[:10]
top_10_meaning_wrong = score_summary_word.loc[score_summary_word['Num Meaning Wrong']>=1].sort_values(by='Num Meaning Wrong', ascending=False)['Word'].to_list()[:10]

In [7]:
# 1) Total quiz attempts by day - line graph with correct/wrong breakdown
quiz_by_date = df.groupby('Last Quiz').agg({
    'Quiz Attempts': 'sum',
    'Num Pinyin Correct': 'sum',
    'Num Meaning Correct': 'sum',
    'Num Pinyin Wrong': 'sum',
    'Num Meaning Wrong': 'sum'
}).reset_index()

quiz_by_date['Total Correct'] = quiz_by_date['Num Pinyin Correct'] + quiz_by_date['Num Meaning Correct']
quiz_by_date['Total Wrong'] = quiz_by_date['Num Pinyin Wrong'] + quiz_by_date['Num Meaning Wrong']

fig1 = go.Figure()
fig1.add_trace(go.Scatter(
    x=quiz_by_date['Last Quiz'], 
    y=quiz_by_date['Total Correct'],
    mode='lines+markers',
    name='Correct',
    line=dict(color='green')
))
fig1.add_trace(go.Scatter(
    x=quiz_by_date['Last Quiz'], 
    y=quiz_by_date['Total Wrong'],
    mode='lines+markers',
    name='Wrong',
    line=dict(color='red')
))
fig1.update_layout(
    title='Quiz Attempts by Day (Correct vs Wrong)',
    xaxis_title='Date',
    yaxis_title='Number of Attempts',
    hovermode='x unified'
)
fig1.show()

In [8]:
# 2) Count of words by category with right/wrong percentage (stacked bar)
category_stats = df.groupby('Word Category').agg({
    'Word': 'count',
    'Num Pinyin Correct': 'sum',
    'Num Meaning Correct': 'sum',
    'Num Pinyin Wrong': 'sum',
    'Num Meaning Wrong': 'sum'
}).reset_index()

category_stats['Total Correct'] = category_stats['Num Pinyin Correct'] + category_stats['Num Meaning Correct']
category_stats['Total Wrong'] = category_stats['Num Pinyin Wrong'] + category_stats['Num Meaning Wrong']
category_stats['Total Attempts'] = category_stats['Total Correct'] + category_stats['Total Wrong']
category_stats['Correct %'] = (category_stats['Total Correct'] / category_stats['Total Attempts'] * 100).fillna(0)
category_stats['Wrong %'] = (category_stats['Total Wrong'] / category_stats['Total Attempts'] * 100).fillna(0)

fig2 = go.Figure()
fig2.add_trace(go.Bar(
    x=category_stats['Word Category'],
    y=category_stats['Correct %'],
    name='Correct %',
    marker_color='green',
    text=category_stats['Correct %'].round(1).astype(str) + '%',
    textposition='inside'
))
fig2.add_trace(go.Bar(
    x=category_stats['Word Category'],
    y=category_stats['Wrong %'],
    name='Wrong %',
    marker_color='red',
    text=category_stats['Wrong %'].round(1).astype(str) + '%',
    textposition='inside'
))
fig2.update_layout(
    title='Quiz Performance by Word Category',
    xaxis_title='Category',
    yaxis_title='Percentage',
    barmode='stack',
    xaxis_tickangle=-45
)
fig2.show()

In [21]:
# 3) Top 10 words with pinyin wrong / meaning wrong AND top 10 correct
from plotly.subplots import make_subplots

# Top 10 Pinyin Wrong by percentage, then by count
top_pinyin = df.groupby('Word').agg({
    'Num Pinyin Wrong': 'sum',
    'Quiz Attempts': 'sum'
}).reset_index()
top_pinyin = top_pinyin[top_pinyin['Quiz Attempts'] > 1]  # More than 1 attempt
top_pinyin['Pinyin Wrong %'] = (top_pinyin['Num Pinyin Wrong'] / top_pinyin['Quiz Attempts'] * 100).round(1)
top_pinyin = top_pinyin[top_pinyin['Num Pinyin Wrong'] >= 1].sort_values(
    by=['Pinyin Wrong %', 'Num Pinyin Wrong'], ascending=[False, False]
).head(10)
top_pinyin['Label'] = '(' + top_pinyin['Pinyin Wrong %'].astype(str) + '%, ' + top_pinyin['Quiz Attempts'].astype(str) + ')'

# Top 10 Meaning Wrong by percentage, then by count
top_meaning = df.groupby('Word').agg({
    'Num Meaning Wrong': 'sum',
    'Quiz Attempts': 'sum'
}).reset_index()
top_meaning = top_meaning[top_meaning['Quiz Attempts'] > 1]  # More than 1 attempt
top_meaning['Meaning Wrong %'] = (top_meaning['Num Meaning Wrong'] / top_meaning['Quiz Attempts'] * 100).round(1)
top_meaning = top_meaning[top_meaning['Num Meaning Wrong'] >= 1].sort_values(
    by=['Meaning Wrong %', 'Num Meaning Wrong'], ascending=[False, False]
).head(10)
top_meaning['Label'] = '(' + top_meaning['Meaning Wrong %'].astype(str) + '%, ' + top_meaning['Quiz Attempts'].astype(str) + ')'

# Top 10 Pinyin Correct by percentage, then by count
top_pinyin_correct = df.groupby('Word').agg({
    'Num Pinyin Correct': 'sum',
    'Quiz Attempts': 'sum'
}).reset_index()
top_pinyin_correct = top_pinyin_correct[top_pinyin_correct['Quiz Attempts'] > 1]  # More than 1 attempt
top_pinyin_correct['Pinyin Correct %'] = (top_pinyin_correct['Num Pinyin Correct'] / top_pinyin_correct['Quiz Attempts'] * 100).round(1)
top_pinyin_correct = top_pinyin_correct[top_pinyin_correct['Num Pinyin Correct'] >= 1].sort_values(
    by=['Pinyin Correct %', 'Num Pinyin Correct'], ascending=[False, False]
).head(10)
top_pinyin_correct['Label'] = '(' + top_pinyin_correct['Pinyin Correct %'].astype(str) + '%, ' + top_pinyin_correct['Quiz Attempts'].astype(str) + ')'

# Top 10 Meaning Correct by percentage, then by count
top_meaning_correct = df.groupby('Word').agg({
    'Num Meaning Correct': 'sum',
    'Quiz Attempts': 'sum'
}).reset_index()
top_meaning_correct = top_meaning_correct[top_meaning_correct['Quiz Attempts'] > 1]  # More than 1 attempt
top_meaning_correct['Meaning Correct %'] = (top_meaning_correct['Num Meaning Correct'] / top_meaning_correct['Quiz Attempts'] * 100).round(1)
top_meaning_correct = top_meaning_correct[top_meaning_correct['Num Meaning Correct'] >= 1].sort_values(
    by=['Meaning Correct %', 'Num Meaning Correct'], ascending=[False, False]
).head(10)
top_meaning_correct['Label'] = '(' + top_meaning_correct['Meaning Correct %'].astype(str) + '%, ' + top_meaning_correct['Quiz Attempts'].astype(str) + ')'

fig3 = make_subplots(rows=2, cols=2, subplot_titles=(
    'Top 10 Pinyin Wrong', 'Top 10 Meaning Wrong',
    'Top 10 Pinyin Correct', 'Top 10 Meaning Correct'
))

# Top row - Wrong
fig3.add_trace(go.Bar(
    y=top_pinyin['Word'],
    x=top_pinyin['Pinyin Wrong %'],
    orientation='h',
    marker_color='coral',
    text=top_pinyin['Label'],
    textposition='outside'
), row=1, col=1)

fig3.add_trace(go.Bar(
    y=top_meaning['Word'],
    x=top_meaning['Meaning Wrong %'],
    orientation='h',
    marker_color='steelblue',
    text=top_meaning['Label'],
    textposition='outside'
), row=1, col=2)

# Bottom row - Correct
fig3.add_trace(go.Bar(
    y=top_pinyin_correct['Word'],
    x=top_pinyin_correct['Pinyin Correct %'],
    orientation='h',
    marker_color='lightgreen',
    text=top_pinyin_correct['Label'],
    textposition='outside'
), row=2, col=1)

fig3.add_trace(go.Bar(
    y=top_meaning_correct['Word'],
    x=top_meaning_correct['Meaning Correct %'],
    orientation='h',
    marker_color='mediumseagreen',
    text=top_meaning_correct['Label'],
    textposition='outside'
), row=2, col=2)

fig3.update_layout(
    title='Top 10 Words by Error Rate and Correct Rate',
    showlegend=False,
    height=900,
    margin=dict(r=100)  # Add right margin for text
)
fig3.update_xaxes(title_text='% Incorrect', row=1, col=1, range=[0, 120])
fig3.update_xaxes(title_text='% Incorrect', row=1, col=2, range=[0, 120])
fig3.update_xaxes(title_text='% Correct', row=2, col=1, range=[0, 120])
fig3.update_xaxes(title_text='% Correct', row=2, col=2, range=[0, 120])
fig3.update_yaxes(autorange='reversed')  # Highest at top
fig3.show()

In [10]:
# 4) Count of Words by Category
words_by_category = df.groupby('Word Category')['Word'].count().reset_index()
words_by_category.columns = ['Word Category', 'Count']
words_by_category = words_by_category.sort_values('Count', ascending=True)

fig4 = go.Figure(go.Bar(
    x=words_by_category['Count'],
    y=words_by_category['Word Category'],
    orientation='h',
    marker_color='teal',
    text=words_by_category['Count'],
    textposition='outside'
))
fig4.update_layout(
    title='Word Count by Category',
    xaxis_title='Number of Words',
    yaxis_title='Category',
    height=600
)
fig4.show()

In [11]:
# 7) Words Added Over Time (cumulative)
df['Added Date'] = pd.to_datetime(df['Added Date'], format='mixed', errors='coerce')
words_over_time = df.groupby(df['Added Date'].dt.date)['Word'].count().reset_index()
words_over_time.columns = ['Date', 'Words Added']
words_over_time['Cumulative'] = words_over_time['Words Added'].cumsum()

fig7 = go.Figure()
fig7.add_trace(go.Scatter(
    x=words_over_time['Date'],
    y=words_over_time['Cumulative'],
    mode='lines+markers',
    name='Total Words',
    fill='tozeroy',
    line=dict(color='royalblue')
))
fig7.update_layout(
    title='Vocabulary Growth Over Time',
    xaxis_title='Date',
    yaxis_title='Total Words in Dictionary'
)
fig7.show()

In [12]:
# 8) Quiz Coverage - Words Quizzed vs Not Quizzed
quizzed = (df['Quiz Attempts'] > 0).sum()
not_quizzed = (df['Quiz Attempts'] == 0).sum()

fig8 = go.Figure(go.Pie(
    labels=['Quizzed', 'Not Quizzed'],
    values=[quizzed, not_quizzed],
    marker_colors=['green', 'lightgray'],
    textinfo='label+percent+value',
    hole=0.4
))
fig8.update_layout(title='Quiz Coverage')
fig8.show()

## Test Visualization Import

In [13]:
from main.visualizations import (
    prepare_df,
    create_quiz_by_date_chart,
    create_category_performance_chart,
    create_top_errors_chart,
    create_words_by_category_chart,
    create_vocabulary_growth_chart,
    create_quiz_coverage_chart
)

In [14]:
# Test prepare_df
df_test = load_dict()
df_prepared = prepare_df(df_test)
print("prepare_df test:")
print(f"  - Num Pinyin Wrong column exists: {'Num Pinyin Wrong' in df_prepared.columns}")
print(f"  - Num Meaning Wrong column exists: {'Num Meaning Wrong' in df_prepared.columns}")
print(f"  - Last Quiz is datetime: {df_prepared['Last Quiz'].dtype}")
print(f"  - Added Date is datetime: {df_prepared['Added Date'].dtype}")
df_prepared.head()

prepare_df test:
  - Num Pinyin Wrong column exists: True
  - Num Meaning Wrong column exists: True
  - Last Quiz is datetime: datetime64[ns]
  - Added Date is datetime: datetime64[ns]


,Word Id,Word,Pinyin,Pinyin Simplified,Meaning,Added Date,Word Category,Word Rarity,Type,Sentence,Sentence Pinyin,Sentence Meaning,Quiz Attempts,Num Pinyin Correct,Num Meaning Correct,Last Quiz,Num Pinyin Wrong,Num Meaning Wrong
0,D000002,包裹,bao1 guo3,bao1 guo3,Package,2024-09-21,Object,Common,Noun,他昨天收到了一个包裹。,Wǒ zuótiān shōudào le bāoguǒ.,I received the package yesterday.,2,0,0,2026-01-27,2,2
1,D000004,不错,bu2 cuo4,bu2 cuo4,Good (More positive than 还好),2024-09-21,Opinion,Common,Adjective,这个电影不错，我们可以一起去看。,Zhè gè diànyǐng bùcuò wǒmen kěyǐ yìqǐ qù kàn,This movie is pretty good; we can go watch it ...,0,0,0,NaT,0,0
2,D000005,不过,bu2 guo4,bu2 guo4,But (Similar to dan4 shi4 but less formal),2024-09-21,Grammar,Common,Grammar,我喜欢这家餐厅，不过价格有点贵。,Wǒ xǐhuān zhè jiā cāntīng búguò jiàgé yǒudiǎn ...,I like this restaurant but the price is a bit ...,0,0,0,NaT,0,0
3,D000006,不太,bu2 tai4,bu2 tai4,not quite,2024-09-21,Degree,Common,Adjuster,不太好,bu2 tai4 hao3,not quite good,0,0,0,NaT,0,0
4,D000007,不仅,bu4 jin3,bu4 jin3,Not only ....,2024-09-21,Grammar,Common,Grammar,他不仅喜欢音乐，还喜欢运动,Tā bùjǐn xǐhuān yīnyuè hái xǐhuān yùndòng.,He not only likes music but also likes sports.,0,0,0,NaT,0,0


In [15]:
# Test create_quiz_by_date_chart
fig_quiz_date = create_quiz_by_date_chart(df_prepared)
print(f"create_quiz_by_date_chart test:")
print(f"  - Returns Figure: {type(fig_quiz_date).__name__}")
print(f"  - Number of traces: {len(fig_quiz_date.data)}")
fig_quiz_date.show()

create_quiz_by_date_chart test:
  - Returns Figure: Figure
  - Number of traces: 2


In [16]:
# Test create_category_performance_chart
fig_category = create_category_performance_chart(df_prepared)
print(f"create_category_performance_chart test:")
print(f"  - Returns Figure: {type(fig_category).__name__}")
print(f"  - Number of traces: {len(fig_category.data)}")
fig_category.show()

create_category_performance_chart test:
  - Returns Figure: Figure
  - Number of traces: 2


In [17]:
# Test create_top_errors_chart
fig_errors = create_top_errors_chart(df_prepared)
print(f"create_top_errors_chart test:")
print(f"  - Returns Figure: {type(fig_errors).__name__}")
print(f"  - Number of traces: {len(fig_errors.data)}")
fig_errors.show()

create_top_errors_chart test:
  - Returns Figure: Figure
  - Number of traces: 2


In [18]:
# Test create_words_by_category_chart
fig_words_cat = create_words_by_category_chart(df_prepared)
print(f"create_words_by_category_chart test:")
print(f"  - Returns Figure: {type(fig_words_cat).__name__}")
print(f"  - Number of traces: {len(fig_words_cat.data)}")
fig_words_cat.show()

create_words_by_category_chart test:
  - Returns Figure: Figure
  - Number of traces: 1


In [19]:
# Test create_vocabulary_growth_chart
fig_growth = create_vocabulary_growth_chart(df_prepared)
print(f"create_vocabulary_growth_chart test:")
print(f"  - Returns Figure: {type(fig_growth).__name__}")
print(f"  - Number of traces: {len(fig_growth.data)}")
fig_growth.show()

create_vocabulary_growth_chart test:
  - Returns Figure: Figure
  - Number of traces: 1


In [20]:
# Test create_quiz_coverage_chart
fig_coverage = create_quiz_coverage_chart(df_prepared)
print(f"create_quiz_coverage_chart test:")
print(f"  - Returns Figure: {type(fig_coverage).__name__}")
print(f"  - Number of traces: {len(fig_coverage.data)}")
fig_coverage.show()

create_quiz_coverage_chart test:
  - Returns Figure: Figure
  - Number of traces: 1
